In [1]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [2]:
params_dict = {'L':5.21,'G':9.81,'M':10.0,'K_d':1e-6, 'K_dummy':1e-4}


u_ini_dict = {'theta':np.deg2rad(5.0)}  # for the initialization problem
u_run_dict = {'f_x':10}  # for the running problem (here initialization and running problem are the same)

#u_ini_dict = {'theta':10}  # for the initialization problem
u_run_dict = {'f_x':10}  # for the running problem (here initialization and running problem are the same)



x_list = ['pos_x','pos_y','v_x','v_y']    # [inductor current, PI integrator]
y_ini_list = ['lam','f_x'] # for the initialization problem
y_run_list = ['lam','theta'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [3]:
dpos_x = v_x
dpos_y = v_y
dv_x = (-2*pos_x*lam + f_x - K_d*v_x)/M
dv_y = (-M*G - 2*pos_y*lam - K_d*v_y)/M   

g_1 = pos_x**2 + pos_y**2 - L**2 -lam*K_dummy
g_2 = -theta + sym.atan2(pos_x,-pos_y)

In [4]:
sys = {'name':'pendulum',
       'params_dict':params_dict,
       'f_list':[dpos_x,dpos_y,dv_x,dv_y],
       'g_list':[g_1,g_2],
       'x_list':x_list,
       'y_ini_list':y_ini_list,
       'y_run_list':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':{'g_1':g_1,'PE':M*G*pos_y,'KE':0.5*M*(v_x**2+v_y**2),'theta':theta}}

sys = db.build(sys)

Compilation time: 8.30 s


In [5]:
sys['jac_ini']

Matrix([
[                           0,                           0,      1,      0,          0,   0],
[                           0,                           0,      0,      1,          0,   0],
[                    -2*lam/M,                           0, -K_d/M,      0, -2*pos_x/M, 1/M],
[                           0,                    -2*lam/M,      0, -K_d/M, -2*pos_y/M,   0],
[                     2*pos_x,                     2*pos_y,      0,      0,   -K_dummy,   0],
[-pos_y/(pos_x**2 + pos_y**2), pos_x/(pos_x**2 + pos_y**2),      0,      0,          0,   0]])

In [6]:
sys['jac_run']

Matrix([
[                           0,                           0,      1,      0,          0,  0],
[                           0,                           0,      0,      1,          0,  0],
[                    -2*lam/M,                           0, -K_d/M,      0, -2*pos_x/M,  0],
[                           0,                    -2*lam/M,      0, -K_d/M, -2*pos_y/M,  0],
[                     2*pos_x,                     2*pos_y,      0,      0,   -K_dummy,  0],
[-pos_y/(pos_x**2 + pos_y**2), pos_x/(pos_x**2 + pos_y**2),      0,      0,          0, -1]])

In [7]:
sys['jac_trap']

Matrix([
[                           1,                           0,          -0.5*Dt,                0,              0,  0],
[                           0,                           1,                0,          -0.5*Dt,              0,  0],
[                1.0*Dt*lam/M,                           0, 0.5*Dt*K_d/M + 1,                0, 1.0*Dt*pos_x/M,  0],
[                           0,                1.0*Dt*lam/M,                0, 0.5*Dt*K_d/M + 1, 1.0*Dt*pos_y/M,  0],
[                     2*pos_x,                     2*pos_y,                0,                0,       -K_dummy,  0],
[-pos_y/(pos_x**2 + pos_y**2), pos_x/(pos_x**2 + pos_y**2),                0,                0,              0, -1]])